In [17]:
#up to date 
import pandas as pd
order = pd.read_csv(".\data\Orders.csv")
returns = pd.read_csv(".\data\Returns.csv")
import re
order['Profit'] = order['Profit'].apply(lambda x: re.sub('[$]', '', x)) #remove $ signs
order['Sales'] = order['Sales'].apply(lambda x: re.sub('[$]', '', x)) #remove $ signs
order['Profit'] = order['Profit'].apply(lambda x: re.sub('[,]', '', x)) #remove , signs
order['Sales'] = order['Sales'].apply(lambda x: re.sub('[,]', '', x)) #remove , signs
order['Profit'] = pd.to_numeric(order['Profit'])
order['Sales'] = pd.to_numeric(order['Sales'])
order["Order.Date"] = pd.to_datetime(order["Order.Date"])
order["Ship.Date"] = pd.to_datetime(order["Ship.Date"])

merged = order.merge(returns, how='outer', left_on= "Order.ID", right_on="Order ID")
merged['Returned'] = ['No' if ans != 'Yes' else 'Yes' for ans in merged['Returned']] #NAN -> no
merged['Returned'] = merged['Returned'].astype('category')

merged['Process.Time'] = merged['Ship.Date'] - merged['Order.Date']
merged['Process.Time'] = merged['Process.Time'].dt.days #removes 'days'

In [18]:
returned_orders = merged.groupby(['Product.ID', 'Process.Time', 'Returned'])['Product.ID'].count()
returned_orders.head()

Product.ID   Process.Time  Returned
FUR-BO-3174  2             No          1
             6             No          1
FUR-BO-3175  0             No          1
             2             No          3
             4             No          3
Name: Product.ID, dtype: int64

In [9]:
merged.columns

Index(['Row.ID', 'Order.ID', 'Order.Date', 'Ship.Date', 'Ship.Mode',
       'Customer.ID', 'Customer.Name', 'Segment', 'Postal.Code', 'City',
       'State', 'Country', 'Region_x', 'Market', 'Product.ID', 'Category',
       'Sub.Category', 'Product.Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping.Cost', 'Order.Priority', 'Returned', 'Order ID',
       'Region_y', 'Process.Time'],
      dtype='object')

In [16]:
final = merged.drop(columns=['Row.ID', 'Order.ID', 'Customer.ID', 'Customer.Name', 'Product.ID', 'Order ID', 'Region_y', 'Product.Name'])
final.head()

,Order.Date,Ship.Date,Ship.Mode,Segment,Postal.Code,City,State,Country,Region_x,Market,Category,Sub.Category,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Returned,Process.Time
0,2014-11-11,2014-11-13,First Class,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,Technology,Phones,221.98,2,0.0,62.15,40.77,High,No,2 days
1,2014-11-11,2014-11-13,First Class,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,Furniture,Bookcases,341.96,2,0.0,54.71,25.27,High,No,2 days
2,2014-02-05,2014-02-07,Second Class,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,Furniture,Chairs,3709.40,9,0.1,-288.77,923.63,Critical,No,2 days
3,2014-02-05,2014-02-07,Second Class,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,Technology,Copiers,344.68,2,0.1,34.42,65.35,Critical,No,2 days
4,2014-02-05,2014-02-07,Second Class,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,Office Supplies,Storage,133.92,5,0.1,-6.03,41.64,Critical,No,2 days


In [15]:
# Differentiate numerical features (minus the target) and categorical features
categorical_features = final.select_dtypes(include = ["object"]).columns
numerical_features = final.select_dtypes(exclude = ["object"]).columns

#numerical_features = numerical_features.drop("SalePrice")
#print("Numerical features : " + str(len(numerical_features)))
#print("Categorical features : " + str(len(categorical_features)))
#train_num = train[numerical_features]
#train_cat = train[categorical_features]
print(categorical_features)

print(numerical_features)

Index(['Ship.Mode', 'Segment', 'City', 'State', 'Country', 'Region_x',
       'Market', 'Category', 'Sub.Category', 'Product.Name', 'Order.Priority'],
      dtype='object')
Index(['Order.Date', 'Ship.Date', 'Postal.Code', 'Sales', 'Quantity',
       'Discount', 'Profit', 'Shipping.Cost', 'Returned', 'Process.Time'],
      dtype='object')


In [ ]:
### Problem 5: Fitting Models
- You can use any binary classification method you have learned so far.
- Use 80/20 training and test splits to build your model. 
- Double check the column types before you fit the model.
- Only include useful features. i.e all the `ID`s should be excluded from your training set.
- Not that there are only less than 5% of the orders have been returned, so you should consider using the `createDataPartition` 
function from `caret` package that does a **stratified** random split of the data. Scikit-learn also has a [StratifiedKfold]
(http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn-model-selection-stratifiedkfold) 
 #function that does similar thing.
- Do forget to `set.seed()` before the spilt to make your result reproducible.
- **Note:** We are not looking for the best tuned model in the lab so dont spend too much time on grid search. Focus on model 
 evaluation and the business use case of each model.

In [ ]:
### Problem 6: Evaluating Models
- What is the best metric to evaluate your model. Is accuracy good for this case?
- Now you have multiple models, which one would you pick? 
- Can you get any clue from the confusion matrix? What is the meaning of precision and recall in this case? Which one do you care 
 the most? How will your model help the manager make decisions?
- **Note:** The last question is open-ended. Your answer could be completely different depending on your understanding of this business 
 problem.

In [ ]:
### Problem 7: Feature Engineering Revisit
- Is there anything wrong with the new feature we generated? How should we fix it?
- ***Hint***: For the real test set, we do not know it will get returned or not.